# CTCを用いた可変長音素認識

これまでの検討では、訓練用データには時間情報付きの音素ラベルが与えられていることが前提でした。しかし、ほとんどの音声データベースには時間情報がなく、発話内容しか与えられていません。これは、音素の時間情報を同定する作業が非常に高コストだからです。

このため、ニューラルネットを用いた音声認識は、これまでは学習時にHMM(隠れマルコフモデル)を用いた従来の音声認識を援用し、はじめに各音素の区間を音声認識によって推定する必要がありました。

これに対し、近年ニューラルネットだけで音声認識を可能とする枠組がいくつか提案されています。CTC (Connectionist Temporal Classification) は、そのようなアルゴリズムの1つです。

CTC は、基本的には LSTM を用いたフレームごとの音素認識に過ぎません。しかし、損失関数が異なります。CTC では、縮約すると正解音素列と一致するような全ての音素列の確率の和をネットワークの出力の「望ましさ」と考え、その対数にマイナスを付けた関数を損失関数とします。例えば、/h a i/という音素列と一致するような長さ5の音素列を全て挙げると

In [162]:
from itertools import combinations, cycle
import numpy as np

def expandstr(str, seqlen):
    l = []
    for expdstrlen in range(len(str), seqlen+1):
        combos = combinations(range(1, expdstrlen), len(str)-1)
        for sp in combos:
            tmp = np.array(sp)
            clens = np.append(tmp,expdstrlen) - np.insert(tmp,0,0)
            bstr = "".join([c * clens[i] for i, c in enumerate(str)])
            bcombos = combinations_with_replacement(range(0,expdstrlen+1), seqlen-expdstrlen)
            # print('blank positions: {}'.format([bcombo for bcombo in bcombos]))
            for bcombo in bcombos:
                xstr = ""
                for i in range(len(bstr)+1):
                    xstr += '_' * bcombo.count(i)
                    if i < len(bstr):
                        xstr += bstr[i]
                l.append(xstr)
    return l

In [163]:
expandstr('hai',5)

['__hai',
 '_h_ai',
 '_ha_i',
 '_hai_',
 'h__ai',
 'h_a_i',
 'h_ai_',
 'ha__i',
 'ha_i_',
 'hai__',
 '_haii',
 'h_aii',
 'ha_ii',
 'hai_i',
 'haii_',
 '_haai',
 'h_aai',
 'ha_ai',
 'haa_i',
 'haai_',
 '_hhai',
 'h_hai',
 'hh_ai',
 'hha_i',
 'hhai_',
 'haiii',
 'haaii',
 'haaai',
 'hhaii',
 'hhaai',
 'hhhai']

In [164]:
bcombo=(0,0)
print(bcombo.count(0))

2


In [165]:
list(combinations(range(1, 5), 3-1))

[(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]

In [166]:
list(combinations_with_replacement(range(0,4),2))

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (1, 1),
 (1, 2),
 (1, 3),
 (2, 2),
 (2, 3),
 (3, 3)]

となります。ただし _ はどの音素でもない記号で、ブランクと呼びます。

以下、ネットワークの定義はLSTMの時と同じです。

In [167]:
import chainer
import chainer.links as L
import chainer.functions as F

class RNN(chainer.Chain):

    def __init__(self, n_lstm_layers=1, n_mid_units=100, n_out=41, dropout=0.2):
        super(RNN, self).__init__()

        # パラメータを持つ層の登録
        with self.init_scope():
            self.l1 = L.Linear(None, n_mid_units)
            self.l2 = L.NStepBiLSTM(n_lstm_layers, n_mid_units, n_mid_units, dropout)
            self.l3 = L.Linear(n_mid_units * 2, n_out)

    def __call__(self, x):
        # データを受け取った際のforward計算を書く
        numframes = [X.shape[0] for X in x]
        split_point = np.cumsum(numframes)[:-1]
        x1 = F.concat(x, axis=0)
        h1 = F.relu(self.l1(x1))
        h1 = F.split_axis(h1, split_point, axis=0)
        hy, cy, ys = self.l2(None, None, h1)
        h2 = F.concat(ys, axis=0)
        return self.l3(h2)

/var/tmp/hiroki/conda/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [168]:
from chainer.dataset import to_device
from chainer.dataset import concat_examples
import numpy as np

def converter(batch, device):
    # alternative to chainer.dataset.concat_examples
    
    Xs = [to_device(device, X) for X, _, __ in batch]
    ts = [to_device(device, t) for _, t, __ in batch]

    lab_batch = [lab.astype(np.int32) for _, __, lab in batch]
    labs = concat_examples(lab_batch, device, padding=0)
    
    return Xs, ts, labs

## 損失関数

CTCの損失関数は、`connectionist_temporal_classification`関数で計算できます。この関数は、これまでのようにフレームごとでなく発話全体に対して計算されます。各フレームの正解音素ラベル `t` を必要とせず、発話全体の音素ラベル列 `lab` さえあればよい点が重要なポイントです。

In [169]:
import numpy as np
from chainer.backends import cuda

def calculate_loss(model, batch, blank_symbol, gpu_id=0, train_mode=True):
    numframes = [X.shape[0] for (X, t, lab) in batch]
    
    # lab を行列に変形
    label_length = xp.array([len(lab) for (X, _t, lab) in batch],dtype=np.int32)
    Xs, _, labs = converter(batch, gpu_id) # アラインメント済ラベルは使わない

    with chainer.using_config('train', train_mode), \
         chainer.using_config('enable_backprop', train_mode):
        ys = net(Xs)
    
    # 発話ごとに分割
    split_point = np.cumsum(numframes)[:-1]
    y4utt = F.split_axis(ys, split_point, axis=0)
    input_length = xp.array(numframes, dtype=np.int32)
    
    # 第iフレームの確率行列(B,V)となるよう整形
    y4utt = F.pad_sequence(y4utt)
    y4frame = F.stack(y4utt, axis=1)
    x = [xi for xi in y4frame]
    
    # ロスの計算
    loss = F.connectionist_temporal_classification(x, labs, blank_symbol, input_length, label_length)

    return loss

In [170]:
import random
import numpy as np
import chainer

def reset_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    if gpu_id >= 0:
        chainer.cuda.cupy.random.seed(seed)

In [171]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

with open('phones') as f:
    phones = f.read().splitlines()
le = LabelEncoder()
le.fit(phones)
blank_symbol = np.asscalar(le.transform(['_'])[0])
nsymbol = len(phones)

In [172]:
import numpy as np
from chainer import iterators
from chainer import optimizers
from chainer.backends import cuda
from chainer.cuda import to_cpu
from chainer.dataset import concat_examples

gpu_id = 0 # CPUを用いる場合は、この値を-1にしてください

batchsize = 100
max_epoch = 200

xp = np
if gpu_id >= 0:
    xp = chainer.cuda.cupy

reset_seed(1234) # 乱数の種をセット

train = np.load("MHT-train.npy") # train: 450 x (framelen x 26 , framelen, maxphonenum)
test = np.load("MHT-test.npy")

train_iter = iterators.SerialIterator(train, batchsize)

# dropoutを大きくしないと、訓練データの音素列に過剰に適合するようだ
net = RNN(n_lstm_layers=1, n_mid_units=200, n_out=nsymbol, dropout=0.4)

#optimizer = optimizers.SGD(lr=0.0001).setup(net)
optimizer = optimizers.Adam().setup(net)

if gpu_id >= 0:
    net.to_gpu(gpu_id)
    
while train_iter.epoch < max_epoch:

    # ---------- 学習の1イテレーション ----------
    train_batch = train_iter.next()

    loss = calculate_loss(net, train_batch, blank_symbol, gpu_id, train_mode = True)

    # 勾配の計算
    net.cleargrads()
    loss.backward()

    # バッチ単位で古い記憶を削除し、計算コストを削減する。
    loss.unchain_backward()

    # パラメータの更新
    optimizer.update()
    
    # --------------- ここまで ----------------

    # 1エポック終了ごとにValidationデータに対する予測精度を測って、
    # モデルの汎化性能が向上していることをチェックしよう
    if train_iter.is_new_epoch:  # 1 epochが終わったら
        # ロスの表示
        print('epoch:{:02d} train_loss:{:.04f} '.format(
            train_iter.epoch, float(to_cpu(loss.data))), end='')

        valid_losses = []
        
        loss_valid = calculate_loss(net, test, blank_symbol, gpu_id, train_mode=False)
        valid_losses.append(to_cpu(loss_valid.array))

        print('val_loss:{:.04f}'.format(np.mean(valid_losses)))

epoch:01 train_loss:541.2222 val_loss:401.2349
epoch:02 train_loss:470.5593 val_loss:428.6360
epoch:03 train_loss:427.5804 val_loss:344.4825
epoch:04 train_loss:421.0086 val_loss:328.5212
epoch:05 train_loss:407.4021 val_loss:335.0801
epoch:06 train_loss:386.1248 val_loss:335.0721
epoch:07 train_loss:396.0114 val_loss:322.8109
epoch:08 train_loss:404.2401 val_loss:316.6742
epoch:09 train_loss:399.6852 val_loss:312.7868
epoch:10 train_loss:383.7752 val_loss:311.9551
epoch:11 train_loss:356.5416 val_loss:308.7933
epoch:12 train_loss:370.9811 val_loss:307.4981
epoch:13 train_loss:336.1074 val_loss:304.7755
epoch:14 train_loss:356.0455 val_loss:299.3780
epoch:15 train_loss:366.9304 val_loss:297.7372
epoch:16 train_loss:351.1081 val_loss:295.9219
epoch:17 train_loss:333.5387 val_loss:298.4402
epoch:18 train_loss:343.1516 val_loss:286.4994
epoch:19 train_loss:327.2886 val_loss:286.1173
epoch:20 train_loss:345.2128 val_loss:287.5588
epoch:21 train_loss:354.4489 val_loss:274.3036
epoch:22 trai

epoch:180 train_loss:4.1837 val_loss:13.9531
epoch:181 train_loss:4.0463 val_loss:13.9105
epoch:182 train_loss:3.9053 val_loss:13.8159
epoch:183 train_loss:4.2023 val_loss:13.9348
epoch:184 train_loss:3.5191 val_loss:13.6672
epoch:185 train_loss:3.8324 val_loss:13.8428
epoch:186 train_loss:4.0660 val_loss:13.7634
epoch:187 train_loss:3.8980 val_loss:13.6361
epoch:188 train_loss:3.8478 val_loss:14.3007
epoch:189 train_loss:3.6572 val_loss:14.0663
epoch:190 train_loss:3.4105 val_loss:13.4339
epoch:191 train_loss:3.4242 val_loss:13.5659
epoch:192 train_loss:3.6747 val_loss:13.9719
epoch:193 train_loss:3.1519 val_loss:13.7959
epoch:194 train_loss:3.2905 val_loss:13.7073
epoch:195 train_loss:2.7839 val_loss:13.4870
epoch:196 train_loss:2.8311 val_loss:14.0571
epoch:197 train_loss:3.0772 val_loss:13.7330
epoch:198 train_loss:2.6498 val_loss:13.7721
epoch:199 train_loss:2.7508 val_loss:13.6288
epoch:200 train_loss:2.6865 val_loss:13.8908


## 音素認識結果の観察

In [173]:
test_utterance_number = 0

if gpu_id >= 0:
    net.to_gpu(gpu_id)

Xs, _, __ = converter(test, gpu_id)

# テストデータを1つ取り出します
X_test = Xs[test_utterance_number]
_, __, lab_test = test[test_utterance_number]

with chainer.using_config('train', False), \
     chainer.using_config('enable_backprop', False):
    y_test = net([X_test])

# Variable形式で出てくるので中身を取り出す
y_test = y_test.array

# 結果をCPUに送る
y_test = to_cpu(y_test)

# 予測確率の最大値のインデックスを見る
pred_label = y_test.argmax(axis=1)

テストデータの0番目 (Jセット第1文) の正解音素列は次の通りです。

In [174]:
le.inverse_transform(lab_test)

/var/tmp/hiroki/conda/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['sil', 'ch', 'i', 'i', 's', 'a', 'n', 'a', 'u', 'n', 'a', 'g', 'i',
       'y', 'a', 'n', 'i', 'pau', 'n', 'e', 'Q', 'k', 'i', 'n', 'o', 'y',
       'o', 'o', 'n', 'a', 'm', 'o', 'n', 'o', 'g', 'a', 'm', 'i', 'n',
       'a', 'g', 'i', 'r', 'u', 'sil'], dtype='<U3')

これに対し、まず各フレームに対する音素認識結果を見てみます。

In [175]:
le.inverse_transform(pred_label)

/var/tmp/hiroki/conda/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['_', '_', '_', '_', '_', '_', '_', '_', 'sil', 'sil', 'sil', '_',
       '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_',
       '_', '_', '_', '_', 'ch', 'ch', 'ch', '_', '_', '_', '_', 'i', 'i',
       'i', 'i', 'i', 'i', 'i', 'i', '_', '_', '_', '_', '_', '_', 's',
       's', 's', '_', '_', 'a', 'a', 'a', '_', '_', '_', '_', 'n', 'n',
       'n', 'a', 'a', '_', '_', '_', '_', '_', '_', '_', '_', 'u', 'u',
       'u', '_', '_', '_', '_', '_', '_', '_', '_', 'n', 'n', 'a', 'a',
       'a', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'g', 'g',
       'g', '_', '_', '_', '_', '_', '_', 'i', 'i', 'i', 'i', 'i', 'y',
       '_', 'a', 'a', 'a', 'a', '_', '_', '_', '_', '_', '_', '_', '_',
       'n', 'n', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'i',
       'i', 'i', '_', '_', '_', '_', '_', 'pau', 'pau', '_', '_', '_',
       '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_',
       '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_',

多くのフレームで _ (ブランク) が出力されているのがわかります。これは、最も「自信がある」フレームに対してだけ具体的な音素が出力され、音素境界に近い曖昧な部分では何も出力していないものと解釈することができます。

繰り返しですが、この結果をLSTM等と単純に比較することはできません。CTCの学習には音素の時間情報を使っていないので、こちらの方が不利だからです。

出力された冗長な音素列を縮約することで、次のように最終的な音素認識結果が得られます。

In [176]:
mask = pred_label[:-1] != pred_label[1:]
tmp_label = pred_label[np.append(mask,True)]
mask = tmp_label != blank_symbol
le.inverse_transform(tmp_label[mask])

/var/tmp/hiroki/conda/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['sil', 'ch', 'i', 's', 'a', 'n', 'a', 'u', 'n', 'a', 'g', 'i', 'y',
       'a', 'n', 'i', 'pau', 'n', 'e', 'Q', 'k', 'i', 'n', 'o', 'y', 'o',
       'o', 'n', 'a', 'm', 'o', 'n', 'o', 'g', 'a', 'n', 'i', 'n', 'a',
       'N', 'i', 'r', 'u', 'sil'], dtype='<U3')